# LLM Chatbot powered with a Vector DB to mitigate HALLUCIANTION problem and enhance the generation of factual information. In this project I'm building a LLM chatbot capable of learning from external world using Retrieval Augmented Generation (RAG). In this project I'm using Langchain, OpenAI, Pinecone, Datasets library.

## **Installing dependencies**

In [1]:
!pip install langchain openai datasets pinecone-client tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
! pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9145e2fbd519b2c0caa523a5df62e2d1571e791380bbd03d95c679a31ff6fd03
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


# **Lets first build LLM Chatbot without RAG**

#### First will start by building a normal chatbot using OpenAI's gpt3.5 turbo. Its knowledge is updated till January 2022, which means this model doesnot have latest world information example: Langchain, but will increase its capability for this certain task only. We will use langchain docs datasets from HuggingFace and store them to Vector DB so that gpt3.5 can get relevants vectors or information from the VectorDB to generate factual outputs. Will try to reduce its HALLUCINATION, by providing context and with relevant information. We can even measure its hallucination by using vectara hallucination evaluation model. They claim that if the value or score is below 0.5 then it means LLM is generating make up results if value is more 0.5 then it means it is generating relevant information. More information about vectara can be found here: https://huggingface.co/vectara/hallucination_evaluation_model


#### Note: To be honest when I used this hall eval model on various hallucinated results it gave score more than 0.5 and on non hallucinated results it gave below 0.5, So we cannot completely rely on this model heavily but it can help us to distinguish

In [3]:
from google.colab import userdata


# Lets import OpenAI API key here

In [4]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo-16k',temperature=0.9
)

# In langchain we pass instructions like this:

In [5]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful, respectful and honest assistant. If you don't know anything its okay to say I don't know."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="Can you tell me what is Transformer?")
]

## Here we received output from GPT 3.5Turbo model

In [6]:
res = chat(messages)
res

AIMessage(content='Certainly! The Transformer is a deep learning model architecture introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. It has been widely used in natural language processing tasks, such as machine translation and language generation.\n\nThe Transformer model relies on the concept of self-attention or scaled dot-product attention, where the model attends to all positions in an input sequence to compute a weighted sum of the representations at each position. This allows the model to capture dependencies between different parts of the sequence effectively.\n\nThe Transformer architecture consists of an encoder and a decoder. The encoder takes an input sequence and produces a sequence of contextualized representations, while the decoder generates an output sequence based on the encoder\'s representations and an attention mechanism.\n\nThe Transformer model has gained attention due to its parallelizability, which accelerates training and inference

In [7]:
res.content

'Certainly! The Transformer is a deep learning model architecture introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. It has been widely used in natural language processing tasks, such as machine translation and language generation.\n\nThe Transformer model relies on the concept of self-attention or scaled dot-product attention, where the model attends to all positions in an input sequence to compute a weighted sum of the representations at each position. This allows the model to capture dependencies between different parts of the sequence effectively.\n\nThe Transformer architecture consists of an encoder and a decoder. The encoder takes an input sequence and produces a sequence of contextualized representations, while the decoder generates an output sequence based on the encoder\'s representations and an attention mechanism.\n\nThe Transformer model has gained attention due to its parallelizability, which accelerates training and inference, and its ability 

In [8]:
messages.append(res)

In [9]:
prompt = HumanMessage(
    content="Why do transformers are so powerful than seq2seq models?"
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

"Transformers have several advantages over traditional seq2seq models, which contribute to their increased power and effectiveness in various natural language processing tasks:\n\n1. Attention mechanism: Transformers leverage self-attention or scaled dot-product attention, allowing them to capture dependencies between different parts of the input sequence effectively. This mechanism allows the model to attend to relevant positions in the sequence and assign different weights to different parts during encoding and decoding. This attention mechanism helps the model understand the context and relationships between different words or tokens in the sequence.\n\n2. Parallel processing: Unlike traditional seq2seq models, which typically process inputs sequentially, Transformers can parallelize computations across multiple positions in the input sequence. This parallel processing capability makes Transformers faster during both training and inference, enabling them to handle large datasets and

# **Now we will ask some latest world questions from our LLM chatbot and will deal with its Halluciantions later**

##### While we are using GPT3.5 turbo model, it's important to note that the knowledge of Large Language Models (LLMs) is constrained. This limitation arises because LLMs acquire their knowledge exclusively during training.

In [10]:
prompt = HumanMessage(
    content="Can you tell me what is llama 2?"
)
messages.append(prompt)
res = chat(messages)

In [11]:
res.content

'I\'m sorry, but I\'m not familiar with "llama 2." It is possible that you might be referring to something specific that I\'m not aware of. Could you please provide more context or clarify your question?'

In [12]:
messages.append(res)

# **Lets Now check our model hallucination score using Vectara Hallucianation Evaluation Model**

In [13]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder

model = CrossEncoder('vectara/hallucination_evaluation_model')
scores = model.predict([res.content])
scores

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

0.30502373

### As we can see our LLM chatbot is generating outputs which it is not being trained on. The score is below 0.5 which conveys that model output is not factual. Recently OpenAI changed the behaviours of their models if they don't know about a certain thing.

In [14]:
prompt = HumanMessage(
    content="""A person on a horse jumps over a broken down airplane then that person went to a diner, ordering an omelette."""
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content


"Thank you for sharing that random piece of information. Is there anything specific you would like to discuss or ask about? I'm here to assist you."

In [15]:
scores = model.predict([res.content])
scores

0.09159545

In [16]:
prompt = HumanMessage(
    content="write source code for question answering using Langchain?"
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content


'I\'m sorry, but as of my knowledge, there is no programming language called "Langchain." If you are referring to a different language or framework, please provide more information so that I can help you accordingly.'

In [17]:
scores = model.predict([res.content])
scores

0.29160017

In [18]:
prompt = HumanMessage(
    content="What do you know about langchain??"
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content


'I apologize for the confusion earlier. Upon further research, I couldn\'t find any information regarding a programming language or framework called "Langchain." It\'s possible that it may be a less popular or niche language, or it could be a term or project that is not widely known.\n\nIf you have any other questions or if there\'s something else I can assist you with, please let me know.'

In [19]:
scores = model.predict([res.content])
scores

0.26853397

In [20]:
prompt = HumanMessage(
    content="Can we use Langchain to build LLMs?"
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

'I apologize for the confusion, but I couldn\'t find any information about a programming language or framework called "Langchain" that specifically relates to building LLMs (Language Models). It\'s possible that "Langchain" may be a lesser-known or niche technology, or it could be a term or project that is not widely known within the field of language modeling.\n\nIf you are interested in building LLMs (Language Models), some commonly used frameworks and libraries include TensorFlow, PyTorch, and Hugging Face\'s Transformers library. These tools provide a range of capabilities and resources for developing and training various types of language models.\n\nIf you have any other questions or need further assistance, please let me know.'

In [21]:
scores = model.predict([res.content])
scores

0.34489682

In [22]:
prompt = HumanMessage(
    content="what were the challenges faced by OpenAI while building GPT 4 turbo model?"
)
messages.append(prompt)
res = chat(messages)
messages.append(res)

In [23]:
res.content


"As an AI language model, my responses are based on publicly available information up until September 2021, and I don't have specific insights into developments or challenges faced by OpenAI while building future models like GPT-4 Turbo. However, I can mention some common challenges that are typically associated with building large language models like GPT:\n\n1. Training data: Gathering and curating a vast amount of high-quality training data is a crucial challenge. The dataset needs to be diverse, representative, and free from biases to ensure the model's generalization and fairness.\n\n2. Computational resources: Training large language models like GPT-4 Turbo requires substantial computational resources, including powerful GPUs or TPUs, as well as distributed computing infrastructure. This demand for resources can pose logistical and cost-related challenges.\n\n3. Model optimization: Designing an efficient architecture and optimizing model performance are significant challenges. Im

In [24]:
scores = model.predict([res.content])
scores

0.41314283

## As we can see all the scores are below 0.5 which signifies that LLM is generating output based on hallcuniation. Now we will start building knowledge base only for a certain task for the scope of this project.

#### There is another way of feeding knowledge into LLMs. It is called source knowledge and it refers to any information fed into the LLM via the prompt or prompt engineering.

#### I used the information from openAI website here: https://openai.com/blog/introducing-gpts

In [25]:
gpt_information = [ """We’re rolling out custom versions of ChatGPT that you can create for a specific purpose—called GPTs. GPTs are a new way for anyone to create a tailored version of ChatGPT to be more helpful in their daily life, at specific tasks, at work, or at home—and then share that creation with others. For example, GPTs can help you learn the rules to any board game, help teach your kids math, or design stickers.
Anyone can easily build their own GPT—no coding is required. You can make them for yourself, just for your company’s internal use, or for everyone. Creating one is as easy as starting a conversation, giving it instructions and extra knowledge, and picking what it can do, like searching the web, making images or analyzing data. Try it out at chat.openai.com/create.
Example GPTs are available today for ChatGPT Plus and Enterprise users to try out including Canva and Zapier AI Actions. We plan to offer GPTs to more users soon.

Learn more about our OpenAI DevDay announcements for new models and developer products.
GPTs let you customize ChatGPT for a specific purpose
Since launching ChatGPT people have been asking for ways to customize ChatGPT to fit specific ways that they use it. We launched Custom Instructions in July that let you set some preferences, but requests for more control kept coming. Many power users maintain a list of carefully crafted prompts and instruction sets, manually copying them into ChatGPT. GPTs now do all of that for you.
The best GPTs will be invented by the community
We believe the most incredible GPTs will come from builders in the community. Whether you’re an educator, coach, or just someone who loves to build helpful tools, you don’t need to know coding to make one and share your expertise.
The GPT Store is rolling out later this month
Starting today, you can create GPTs and share them publicly. Later this month, we’re launching the GPT Store, featuring creations by verified builders. Once in the store, GPTs become searchable and may climb the leaderboards. We will also spotlight the most useful and delightful GPTs we come across in categories like productivity, education, and “just for fun”. In the coming months, you’ll also be able to earn money based on how many people are using your GPT.
We built GPTs with privacy and safety in mind
As always, you are in control of your data with ChatGPT. Your chats with GPTs are not shared with builders. If a GPT uses third party APIs, you choose whether data can be sent to that API. When builders customize their own GPT with actions or knowledge, the builder can choose if user chats with that GPT can be used to improve and train our models. These choices build upon the existing privacy controls users have, including the option to opt your entire account out of model training.
We’ve set up new systems to help review GPTs against our usage policies. These systems stack on top of our existing mitigations and aim to prevent users from sharing harmful GPTs, including those that involve fraudulent activity, hateful content, or adult themes. We’ve also taken steps to build user trust by allowing builders to verify their identity. We'll continue to monitor and learn how people use GPTs and update and strengthen our safety mitigations. If you have concerns with a specific GPT, you can also use our reporting feature on the GPT shared page to notify our team.
GPTs will continue to get more useful and smarter, and you’ll eventually be able to let them take on real tasks in the real world. In the field of AI, these systems are often discussed as “agents”. We think it’s important to move incrementally towards this future, as it will require careful technical and safety work—and time for society to adapt. We have been thinking deeply about the societal implications and will have more analysis to share soon.
Developers can connect GPTs to the real world
In addition to using our built-in capabilities, you can also define custom actions by making one or more APIs available to the GPT. Like plugins, actions allow GPTs to integrate external data or interact with the real-world. Connect GPTs to databases, plug them into emails, or make them your shopping assistant. For example, you could integrate a travel listings database, connect a user’s email inbox, or facilitate e-commerce orders.
The design of actions builds upon insights from our plugins beta, granting developers greater control over the model and how their APIs are called. Migrating from the plugins beta is easy with the ability to use your existing plugin manifest to define actions for your GPT.
Enterprise customers can deploy internal-only GPTs
Since we launched ChatGPT Enterprise a few months ago, early customers have expressed the desire for even more customization that aligns with their business. GPTs answer this call by allowing you to create versions of ChatGPT for specific use cases, departments, or proprietary datasets. Early customers like Amgen, Bain, and Square are already leveraging internal GPTs to do things like craft marketing materials embodying their brand, aid support staff with answering customer questions, or help new software engineers with onboarding.
Enterprises can get started with GPTs on Wednesday. You can now empower users inside your company to design internal-only GPTs without code and securely publish them to your workspace. The admin console lets you choose how GPTs are shared and whether external GPTs may be used inside your business. Like all usage on ChatGPT Enterprise, we do not use your conversations with GPTs to improve our models.
We want more people to shape how AI behaves
We designed GPTs so more people can build with us. Involving the community is critical to our mission of building safe AGI that benefits humanity. It allows everyone to see a wide and varied range of useful GPTs and get a more concrete sense of what’s ahead. And by broadening the group of people who decide 'what to build' beyond just those with access to advanced technology it's likely we'll have safer and better aligned AI. The same desire to build with people, not just for them, drove us to launch the OpenAI API and to research methods for incorporating democratic input into AI behavior, which we plan to share more about soon.
We’ve made ChatGPT Plus fresher and simpler to use
Finally, ChatGPT Plus now includes fresh information up to April 2023. We’ve also heard your feedback about how the model picker is a pain. Starting today, no more hopping between models; everything you need is in one place. You can access DALL·E, browsing, and data analysis all without switching. You can also attach files to let ChatGPT search PDFs and other document types. Find us at chatgpt.com. Source : https://openai.com/blog/introducing-gpts """ ]




In [29]:
query = "What do you know about custom versions of ChatGPT?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{gpt_information}

Query: {query}"""

In [30]:
prompt = HumanMessage(
    content=augmented_prompt
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content


"Custom versions of ChatGPT, called GPTs, are being introduced by OpenAI. They provide a way for users to create their own tailored versions of ChatGPT for specific purposes. GPTs are designed to be more helpful in users' daily lives, at work, at home, or for specific tasks.\n\nCreating a custom GPT does not require coding skills. Users can easily build their own GPTs, whether it's for personal use, internal use within their company, or for sharing with others. The process involves starting a conversation, providing instructions and additional knowledge, and specifying the capabilities of the GPT, such as web searching, image generation, or data analysis.\n\nOpenAI highlights that the best GPTs will be created by the community, including educators, coaches, or anyone with expertise and a desire to build helpful tools. The goal is to involve a wide range of people in shaping AI systems and making them more useful for society.\n\nOpenAI is launching the GPT Store, where users can publicl

In [31]:
scores = model.predict([res.content])
scores

0.589675

#### Perfect Our LLM chatbot now knows about Custom GPT and how it can be used for day-today tasks. By giving a knowledge base and prompt our LLM was able to generate factual information and score also tells us its significance.

### Now we know our chatbot has no information about Langchain now we will create a Knowledge base for our chatbot so that our chat bot can refer to knowledge base before answering any question based on Langchain. We will use Ela279/langchain_docs dataset to dump all langchain information in vector database. Our next task is to transform that dataset into the knowledge base that our chatbot can use. To do this we must use an embedding model and vector database.

In [32]:
from datasets import load_dataset

dataset = load_dataset(
    "Ela279/langchain_docs",
    split="train"
)

dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'text', 'source'],
    num_rows: 4152
})

In [33]:
dataset[0]

{'id': '7c3fef993779-0',
 'text': '.rst\n.pdf\nWelcome to LangChain\n Contents \nGetting Started\nModules\nUse Cases\nReference Docs\nEcosystem\nAdditional Resources\nWelcome to LangChain#\nLangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model, but will also be:\nData-aware: connect a language model to other sources of data\nAgentic: allow a language model to interact with its environment\nThe LangChain framework is designed around these principles.\nThis is the Python specific portion of the documentation. For a purely conceptual guide to LangChain, see here. For the JavaScript documentation, see here.\nGetting Started#\nHow to get started using LangChain to create an Language Model application.\nQuickstart Guide\nConcepts and terminology.\nConcepts and terminology\nTutorials created by community experts and presented on YouTube.\nTutorials\nModules#\

# **I'm using Pinecone vector database to store all the vector embedding and query the relevant vectors**

In [34]:
import pinecone

pinecone.init(
    api_key=userdata.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
)

In [35]:
import time

index_name = 'langchain'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pinecone.Index(index_name)


In [36]:
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

#### Here im using openAI embedding model to convert text into vector embeddings

In [37]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [38]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

In [39]:
from tqdm.auto import tqdm

data = dataset.to_pandas()
batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)

    batch = data.iloc[i:i_end]

    ids = [x['id'] for _, x in batch.iterrows()]

    texts = [x['text'] for _, x in batch.iterrows()]

    embeds = embed_model.embed_documents(texts)

    metadata = [
        {'text': x['text'],
         'source': x['source']} for i, x in batch.iterrows()
    ]

    index.upsert(vectors=zip(ids, embeds, metadata))


  0%|          | 0/42 [00:00<?, ?it/s]

In [41]:
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.04152,
 'namespaces': {'': {'vector_count': 4152}},
 'total_vector_count': 4152}

# **Retrieval Augmented Generation**

Now We've built a fully-fledged knowledge vector database. Now it's time to connect that knowledge base to our chatbot. To do that we'll be diving back into LangChain and reusing our template prompt from earlier.

In [42]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Using this vectorstore we can already query the index and see if we have any relevant information given our question about Langchain.

In [43]:
vectorstore

Lets perform a similarity search query and retrieve the relevant results from vector DB

In [44]:
query = "What is so special about Langchain?"
vectorstore.similarity_search(query, k=3)

[Document(page_content='>>> CONTEXT: LangChain: Software. LangChain is a software development framework designed to simplify the creation of applications using large language models. LangChain Initial release date: October 2022. LangChain Programming languages: Python and JavaScript. LangChain Developer(s): Harrison Chase. LangChain License: MIT License. LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only ... Type: Software framework. At its core, LangChain is a framework built around LLMs. We can use it for chatbots, Generative Question-Answering (GQA), summarization, and much more. LangChain is a powerful tool that can be used to work with Large Language Models (LLMs). LLMs are very general in nature, which means that while they can ... LangChain is an intuitive framework created to assist in developing applications driven by a language model, such as OpenAI or Hugging Face. L

# Now we connect our vectorstore to chatbot just like before as we did with custom GPT

In [45]:
def augment_prompt(query: str):

    results = vectorstore.similarity_search(query, k=3)

    source_knowledge = "\n".join([x.page_content for x in results])

    augmented_prompt = f"""Use the relevant information from contexts below and answer the given query.

    Contexts: {source_knowledge}

    Query: {query} """
    return augmented_prompt

In [46]:
print(augment_prompt(query))


Use the relevant information from contexts below and answer the given query.

    Contexts: >>> CONTEXT: LangChain: Software. LangChain is a software development framework designed to simplify the creation of applications using large language models. LangChain Initial release date: October 2022. LangChain Programming languages: Python and JavaScript. LangChain Developer(s): Harrison Chase. LangChain License: MIT License. LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only ... Type: Software framework. At its core, LangChain is a framework built around LLMs. We can use it for chatbots, Generative Question-Answering (GQA), summarization, and much more. LangChain is a powerful tool that can be used to work with Large Language Models (LLMs). LLMs are very general in nature, which means that while they can ... LangChain is an intuitive framework created to assist in developing applic

# Now we have connected our LLM chatbot with knowledge base lets now ask questions about langchain.

In [47]:
prompt = HumanMessage(
    content=augment_prompt(query)
)

res = chat([prompt])

res.content

"LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs). It is a powerful tool that enables developers to build LLM-powered applications more easily. Some notable aspects of LangChain include:\n\n1. Simplified Development: LangChain provides an intuitive and modular framework that assists in developing applications driven by language models. It offers an easy-to-use interface for working with LLMs, making it accessible to developers.\n\n2. Support for Various Applications: LangChain is versatile and can be used for different NLP tasks such as chatbots, Generative Question-Answering (GQA), summarization, and more. It simplifies the process of building advanced language model applications.\n\n3. Language Model Integration: LangChain connects with popular language models like OpenAI or Hugging Face. It provides out-of-the-box support to build NLP applications using LLMs, making it easier to leverage the capabiliti

In [48]:
scores = model.predict([res.content])
scores

0.97827566

# Perfect outcome with a perfect score.

# **No RAG here**

In [66]:
prompt = HumanMessage(
    content="How to load Large Language Models LLMs from langchain ? And Do you know what does Conversational Agent do in LangChain?"
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

'I apologize for any confusion, but based on my knowledge, there is no widely known programming language or framework called "Langchain" that specifically relates to loading Large Language Models (LLMs) or includes a concept of a "Conversational Agent" within that context. It\'s possible that "Langchain" may refer to a specific, lesser-known language or framework that I\'m not familiar with.\n\nIf you can provide more information or clarify your question, I will do my best to assist you. Otherwise, I recommend researching or referring to relevant documentation or resources specific to the "Langchain" framework or language you are referring to.'

In [67]:
scores = model.predict([res.content])
scores

0.5132335

### As we can see here, we are not providing any RAG to our LLM chatbot so it clearly mentioned that it doesnot have any information based on it now we will use RAG to make our LLM chatbot to have the required context to generate the factual information and will meassures its score and the significance of the model.

# **RAG here**

In [68]:
prompt = HumanMessage(
    content=augment_prompt("write source code for question answering using Langchain?")
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

'To write source code for question answering using LangChain, you can follow the steps below:\n\n1. Prepare the Data:\n   - Fetch or load the documents that contain the information you want to use for question answering.\n   - Split the text into smaller chunks if necessary.\n\n2. Set up the VectorStore:\n   - Choose and instantiate a vector store, such as Chroma, to index and search the text data.\n   - Use embeddings, like OpenAIEmbeddings or CohereEmbeddings, to transform the text into vector representations.\n   - Configure the vector store with the indexed texts and embeddings.\n\n3. Perform Similarity Search:\n   - Define a query or question for which you want to find answers.\n   - Use the vector store to perform similarity search and retrieve the most relevant documents or chunks that match the query.\n\n4. Utilize the QA with Sources Chain:\n   - Load the QA with Sources Chain from the langchain.chains.qa_with_sources module.\n   - Use the chain to process the retrieved docume

In [69]:
scores = model.predict([res.content])
scores

0.49197415

In [70]:
prompt = HumanMessage(
    content=augment_prompt(
        "What do you know about langchain??"))

messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

'LangChain is a software development framework designed to simplify the creation of applications using Large Language Models (LLMs). It is an intuitive framework that assists developers in building applications driven by language models like OpenAI or Hugging Face. It provides a standard interface for chains, allowing developers to create sequences of calls that go beyond a single LLM call. With LangChain, developers can work with LLMs to build a variety of applications such as chatbots, generative question answering, summarization, and more.\n\nLangChain allows developers to connect to any model, ingest custom databases, and take action using a framework that abstracts the core building blocks of LLM applications. It offers an open-source and modular approach to developing AI-native applications.\n\nKey features and information about LangChain include:\n- LangChain simplifies embedding creation and storage using tools like Pinecone and Chroma.\n- It supports various programming langua

In [71]:
scores = model.predict([res.content])
scores

0.89464784

In [72]:
prompt = HumanMessage(
    content=augment_prompt(
        "Can you tell me what does Conversational Agent does in LangChain?"))

messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

"In LangChain, a Conversational Agent is a system that utilizes a language model to interact with other tools or agents. Conversational Agents are designed to make decisions, take actions, observe the results, and repeat the process until a desired outcome is achieved. LangChain provides a standard interface for agents, a selection of agents to choose from, and examples of end-to-end agents.\n\nConversational Agents in LangChain enable the development of AI systems that engage in conversations with users or applications. These agents leverage the capabilities of large language models (LLMs) to understand and respond to user queries, interact with APIs, perform grounded question/answering tasks, and take actions based on the context of the conversation.\n\nLangChain's framework for Conversational Agents allows developers to build sophisticated systems that utilize LLMs to understand user intents, generate context-aware responses, and carry out complex interactions. With Conversational A

In [73]:
scores = model.predict([res.content])
scores

0.9890716

In [74]:
prompt = HumanMessage(
    content=augment_prompt(
        "How can we use StochasticAI in Langchain?"))

messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

'To use StochasticAI in LangChain, you can follow the steps outlined below:\n\n1. Installation and Setup:\n   - Install the StochasticAI package using pip: `pip install stochasticx`.\n   - Obtain the StochasticAI API key and set it as an environment variable named `STOCHASTICAI_API_KEY`.\n\n2. Import the StochasticAI Wrapper:\n   - Import the StochasticAI LLM wrapper with the following code: `from langchain.llms import StochasticAI`.\n\n3. Initialize the StochasticAI Object:\n   - Create an instance of the StochasticAI language model by passing the API URL to the constructor. You can store the API URL in a variable named `YOUR_API_URL`.\n   - Example code:\n     ```python\n     from langchain.llms import StochasticAI\n     \n     llm = StochasticAI(api_url=YOUR_API_URL)\n     ```\n\n4. Create an LLMChain with StochasticAI:\n   - Use a PromptTemplate to define your prompt for interacting with the StochasticAI language model.\n   - Example code:\n     ```python\n     from langchain impor

In [75]:
scores = model.predict([res.content])
scores

0.9500458

In [76]:
prompt = HumanMessage(
    content=augment_prompt(
        "what is some special things we can do in Langchain?"))

messages.append(prompt)
res = chat(messages)
messages.append(res)
res.content

"In LangChain, there are several special things that you can do to leverage its capabilities:\n\n1. Personal Assistants: LangChain is well-suited for creating personal assistants. It allows you to develop AI-driven applications that can take actions, remember interactions, and have knowledge about your data. By utilizing LangChain, you can build powerful personal assistants that assist users in various tasks.\n\n2. Question Answering: LangChain excels in answering questions over specific documents by utilizing the information contained within those documents. It enables you to construct accurate answers based on the content of the documents, making it a valuable tool for question-answering tasks.\n\n3. Chatbots: With LangChain's language model integration, you can easily create chatbots. Language models, which are good at generating text, can be combined with LangChain to build chatbots that engage in conversational interactions and provide informative and contextually relevant respons

In [77]:
scores = model.predict([res.content])
scores

0.7758243

In [ ]:
#pinecone.delete_index(index_name)


# As we can see after using RAG our LLM chatbot is giving perfect and relevant information whose score is also more than 0.5 which signifies that all the generated information are significant and based on actual facts. SO this is it,this was an overall solution to build a Hallucination free LLM chatbot.  